# Utilizing a GPU with Keras / TF 2 on MacOS

Herein lies my experience trying to get a functional DL stack working on MacOS Catalina. I started with [this](https://towardsdatascience.com/gpu-accelerated-machine-learning-on-macos-48d53ef1b545) guide.

## Installing a CUDA-Alternative Backend for Keras

The guide suggests that using [PlaidML](https://github.com/plaidml/plaidml) as the CUDA-alternative computational framework. So we will begin by making a new `conda` environment for doing deep learning:
```bash
conda create --name DL python=3
conda activate DL
pip install plaidml-keras
plaidml-setup
```
    
Then, as in the guide, you should see something like this:
```terminal
PlaidML Setup (0.7.0)

Thanks for using PlaidML!

The feedback we have received from our users indicates an ever-increasing need
for performance, programmability, and portability. During the past few months,
we have been restructuring PlaidML to address those needs.  To make all the
changes we need to make while supporting our current user base, all development
of PlaidML has moved to a branch — plaidml-v1. We will continue to maintain and
support the master branch of PlaidML and the stable 0.7.0 release.

Read more here: https://github.com/plaidml/plaidml

Some Notes:
  * Bugs and other issues: https://github.com/plaidml/plaidml/issues
  * Questions: https://stackoverflow.com/questions/tagged/plaidml
  * Say hello: https://groups.google.com/forum/#!forum/plaidml-dev
  * PlaidML is licensed under the Apache License 2.0


Default Config Devices:
   llvm_cpu.0 : CPU (via LLVM)
   metal_intel(r)_uhd_graphics_630.0 : Intel(R) UHD Graphics 630 (Metal)
   metal_amd_radeon_pro_5500m.0 : AMD Radeon Pro 5500M (Metal)

Experimental Config Devices:
   llvm_cpu.0 : CPU (via LLVM)
   opencl_amd_radeon_pro_5500m_compute_engine.0 : AMD AMD Radeon Pro 5500M Compute Engine (OpenCL)
   opencl_intel_uhd_graphics_630.0 : Intel Inc. Intel(R) UHD Graphics 630 (OpenCL)
   metal_intel(r)_uhd_graphics_630.0 : Intel(R) UHD Graphics 630 (Metal)
   metal_amd_radeon_pro_5500m.0 : AMD Radeon Pro 5500M (Metal)

Using experimental devices can cause poor performance, crashes, and other nastiness.

Enable experimental device support? (y,n)[n]:n

Multiple devices detected (You can override by setting PLAIDML_DEVICE_IDS).
Please choose a default device:

   1 : llvm_cpu.0
   2 : metal_intel(r)_uhd_graphics_630.0
   3 : metal_amd_radeon_pro_5500m.0

Default device? (1,2,3)[1]:3

Selected device:
    metal_amd_radeon_pro_5500m.0

Almost done. Multiplying some matrices...
Tile code:
  function (B[X,Z], C[Z,Y]) -> (A) { A[x,y : X,Y] = +(B[x,z] * C[z,y]); }
Whew. That worked.

Save settings to /Users/nicholasrenninger/.plaidml? (y,n)[y]:y
Success!
```
    
**Warning from the guide:**

*This library supports many but not all Keras layers. If your architecture involves only Dense, LSTM, CNN and Dropout layers, you’re certainly good, otherwise check the documentation.*

Now, we just pre-pend the following lines of code to your program to activate the PlaidML backend:
```python
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
```

Now, to test all of this, we try the following [example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the Keras documentation:

In [5]:
import os

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 15s 248us/step - loss: 0.2786 - acc: 0.9144 - val_loss: 0.0578 - val_acc: 0.9807
Epoch 2/12
60000/60000 [==============================] - 11s 184us/step - loss: 0.0927 - acc: 0.9731 - val_loss: 0.0453 - val_acc: 0.9845
Epoch 3/12
60000/60000 [==============================] - 11s 188us/step - loss: 0.0696 - acc: 0.9797 - val_loss: 0.0409 - val_acc: 0.9868
Epoch 4/12
60000/60000 [==============================] - 11s 189us/step - loss: 0.0565 - acc: 0.9830 - val_loss: 0.0317 - val_acc: 0.9885
Epoch 5/12
60000/60000 [==============================] - 11s 187us/step - loss: 0.0488 - acc: 0.9856 - val_loss: 0.0325 - val_acc: 0.9888
Epoch 6/12
60000/60000 [==============================] - 11s 186us/step - loss: 0.0437 - acc: 0.9867 - val_loss: 0.0276 - val_acc: 0.9904
Epoch 7/12
60000/60000 [===================